## Projet Final - Optimisation des Reservoirs Sous Incertitudes Implementation Numerique

Alice Ferry 

#### Importing necessary packages 

In [11]:
#import Pkg
#Pkg.add("JuMP")
#Pkg.add("Gurobi")

In [12]:
using JuMP
using Gurobi
using Random
using Statistics
using Printf

### Generation des donnees 

Toutes les données ont été choisies de manière plus ou moins arbitraire, mais essaie de représenter la réalité le mieux possible.

In [13]:
Random.seed!(21)

T = 5           #timeline de 5 mois
S = 50          #nombre de scenarios testes

#reservoirs 
S_min, S_max = 10.0, 100.0
R_min, R_max = 0.0, 50.0
S_0      = 50.0
S_objectif = 30.0

#couts
C_X = 10.0    #eau relachee planifier
C_Y = 20.0    #cout de l'ajustement
M   = 1000.0  #forte penalite lorsque la demande n'est pas remplies

#parametre statisqieu de la distribution 
mu_I, sigma_I = 20.0, 5.0
mu_D, sigma_D = 25.0, 5.0

I_scenarios = max.(0, mu_I .+ sigma_I .* randn(T, S))
D_scenarios = max.(0, mu_D .+ sigma_D .* randn(T, S))

#pour probleme deterministe
I_mean = fill(mu_I, T)
D_mean = fill(mu_D, T)


5-element Vector{Float64}:
 25.0
 25.0
 25.0
 25.0
 25.0

### Formulation Deterministe (EV)

In [14]:
model_ev = Model(Gurobi.Optimizer)
set_silent(model_ev)


@variable(model_ev, X_ev[1:T] >= 0)
@variable(model_ev, u_ev[1:T] >= 0)
@variable(model_ev, v_ev[1:T] >= 0)
@variable(model_ev, d_ev[1:T] >= 0)
@variable(model_ev, S_ev[1:T])

for t in 1:T
    prev = (t==1) ? S_0 : S_ev[t-1]
    #utilisation des valeurs de l'esperance des variables 
    @constraint(model_ev, S_ev[t] == prev + I_mean[t] - (X_ev[t] + u_ev[t] - v_ev[t]))
    @constraint(model_ev, (X_ev[t] + u_ev[t] - v_ev[t]) + d_ev[t] >= D_mean[t])
    @constraint(model_ev, S_min <= S_ev[t] <= S_max)
    @constraint(model_ev, R_min <= X_ev[t] + u_ev[t] - v_ev[t] <= R_max)
end
@constraint(model_ev, S_ev[T] >= S_objectif)

@objective(model_ev, Min, sum(C_X*X_ev[t] + C_Y*(u_ev[t]+v_ev[t]) + M*d_ev[t] for t in 1:T))

optimize!(model_ev)

X_bar = value.(X_ev) 
println("Formulation deterministe: ", round.(X_bar, digits=2))

Set parameter Username
Academic license - for non-commercial use only - expires 2026-11-19
Formulation deterministe: [20.0, 25.0, 25.0, 25.0, 25.0]


### Methode 1: SAA 

La formulation du probleme est issue de celle ecrite dans le pdf.

In [15]:
#parametres
Random.seed!(21) 

T = 5           
S = 20          
mu_I, sigma_I = 20.0, 5.0
mu_D, sigma_D = 25.0, 5.0

I_scenarios = max.(0, mu_I .+ sigma_I .* randn(T, S))
D_scenarios = max.(0, mu_D .+ sigma_D .* randn(T, S))


S_min, S_max = 10.0, 100.0
R_min, R_max = 0.0, 50.0
S_0          = 50.0   
S_objectif   = 30.0   

C_X = 10.0    
C_Y = 20.0    
M   = 1000.0  

println("Resolution du Probleme Stochastique (SAA)")

model = Model(Gurobi.Optimizer)
set_silent(model) 

#variable de premiere etape
@variable(model, X[1:T] >= 0)

#variable de deuxiemme etape
@variable(model, u[1:T, 1:S] >= 0)      #ajustement a la hausse
@variable(model, v[1:T, 1:S] >= 0)      #ajustement a la baisse
@variable(model, d[1:T, 1:S] >= 0)      #shortage
@variable(model, Stockage[1:T, 1:S])    #niveau du reservoir

#contraintes
for s in 1:S
    for t in 1:T
        
        prev_S = (t == 1) ? S_0 : Stockage[t-1, s]
        @constraint(model, Stockage[t, s] == prev_S + I_scenarios[t, s] - (X[t] + u[t, s] - v[t, s]))
        @constraint(model, (X[t] + u[t, s] - v[t, s]) + d[t, s] >= D_scenarios[t, s])
        @constraint(model, S_min <= Stockage[t, s] <= S_max)
        current_release = X[t] + u[t, s] - v[t, s]
        @constraint(model, R_min <= current_release <= R_max)
    end
    
    @constraint(model, Stockage[T, s] >= S_objectif)
end

#OF
cout_planifie = sum(C_X * X[t] for t in 1:T)

cout_recours = sum((1/S) * (C_Y * (u[t, s] + v[t, s]) + M * d[t, s]) for t in 1:T, s in 1:S)

@objective(model, Min, cout_planifie + cout_recours)

#resolutions
optimize!(model)

if termination_status(model) == MOI.OPTIMAL
    
    global RP = objective_value(model)
    global X_bar = value.(X) #pour EEV
    
    println("Cout Stochastique : ", round(RP, digits=2))
    println("Relachement planifie optimal:")
    println(round.(X_bar, digits=2))
    
    avg_deficit = mean(value.(d))
    println("\nDeficit moyen par periode : ", round(avg_deficit, digits=4))
    
    
    #FOR PAPER 
    println("\nExemple: ")
    println("Temps| Apports| Demande| Planifiee(X)|  Adj(Y)| Déficit| Stock")
    for t in 1:T
        inf = I_scenarios[t, 1]
        dem = D_scenarios[t, 1]
        x_res = value(X[t])
        adj_res = value(u[t, 1]) - value(v[t, 1])
        short_res = value(d[t, 1])
        stor_res = value(Stockage[t, 1])
        
        @printf("%4d | %6.2f | %6.2f | %6.2f | %6.2f | %6.2f | %6.2f\n", 
                t, inf, dem, x_res, adj_res, short_res, stor_res)
    end
else
    println("Probleme non optimal: ", termination_status(model))
end

Resolution du Probleme Stochastique (SAA)
Set parameter Username
Academic license - for non-commercial use only - expires 2026-11-19
Cout Stochastique : 10613.39
Relachement planifie optimal:
[18.15, 19.76, 22.55, 21.41, 23.55]

Deficit moyen par periode : 1.8589

Exemple: 
Temps| Apports| Demande| Planifiee(X)|  Adj(Y)| Déficit| Stock
   1 |  19.86 |  27.40 |  18.15 |   9.25 |   0.00 |  42.46
   2 |  17.28 |  19.76 |  19.76 |   0.00 |   0.00 |  39.98
   3 |  24.82 |  11.85 |  22.55 |   0.00 |   0.00 |  42.25
   4 |  14.25 |  23.77 |  21.41 |   2.36 |   0.00 |  32.73
   5 |  22.49 |  24.58 |  23.55 |   1.03 |   0.00 |  30.64


### Le probleme deterministe est-il plus efficace que le probleme stochastique ? 

In [16]:
#ev
modele_ev = Model(Gurobi.Optimizer)
set_silent(modele_ev)

# Variables (Déterministes, pas de scénarios 's')
@variable(modele_ev, X_ev[1:T] >= 0)
@variable(modele_ev, u_ev[1:T] >= 0)
@variable(modele_ev, v_ev[1:T] >= 0)
@variable(modele_ev, d_ev[1:T] >= 0)
@variable(modele_ev, S_ev[1:T])

for t in 1:T
    prev = (t==1) ? S_0 : S_ev[t-1]
    @constraint(modele_ev, S_ev[t] == prev + mu_I - (X_ev[t] + u_ev[t] - v_ev[t]))
    @constraint(modele_ev, (X_ev[t] + u_ev[t] - v_ev[t]) + d_ev[t] >= mu_D)
    @constraint(modele_ev, S_min <= S_ev[t] <= S_max)
    @constraint(modele_ev, R_min <= X_ev[t] + u_ev[t] - v_ev[t] <= R_max)
end
@constraint(modele_ev, S_ev[T] >= S_objectif)

@objective(modele_ev, Min, sum(C_X*X_ev[t] + C_Y*(u_ev[t]+v_ev[t]) + M*d_ev[t] for t in 1:T))

optimize!(modele_ev)

X_bar = value.(X_ev)
println("Plan Deterministe: ", round.(X_bar, digits=2))


modele_eev = Model(Gurobi.Optimizer)
set_silent(modele_eev)

@variable(modele_eev, u_eev[1:T, 1:S] >= 0)
@variable(modele_eev, v_eev[1:T, 1:S] >= 0) 
@variable(modele_eev, d_eev[1:T, 1:S] >= 0) 
@variable(modele_eev, Stock_eev[1:T, 1:S]) 

for s in 1:S
    for t in 1:T
        stock_prec = (t==1) ? S_0 : Stock_eev[t-1, s]
        
        #moyenne de l etape 1 
        @constraint(modele_eev, Stock_eev[t,s] == stock_prec + I_scenarios[t,s] - (X_bar[t] + u_eev[t,s] - v_eev[t,s]))
        @constraint(modele_eev, (X_bar[t] + u_eev[t,s] - v_eev[t,s]) + d_eev[t,s] >= D_scenarios[t,s])
        @constraint(modele_eev, S_min <= Stock_eev[t,s] <= S_max)
        @constraint(modele_eev, R_min <= X_bar[t] + u_eev[t,s] - v_eev[t,s] <= R_max)
    end
    @constraint(modele_eev, Stock_eev[T,s] >= S_objectif)
end

cout_planifie = sum(C_X * X_bar[t] for t in 1:T)
cout_recours_moyen = (1/S) * sum(C_Y * (u_eev[t,s] + v_eev[t,s]) + M * d_eev[t,s] for t in 1:T, s in 1:S)

@objective(modele_eev, Min, cout_planifie + cout_recours_moyen)

optimize!(modele_eev)
EEV = objective_value(modele_eev)
println("Cout EEV : ", round(EEV, digits=2))


if @isdefined(RP) #vient de SAA 
    VSS = EEV - RP
    
    #format des solutions utilisation AI
    println("\n" * "-"^40)
    println("RÉSULTATS DE LA VSS")
    println("-"^40)
    println("Cout Solution Stochastique: \$", round(RP, digits=2))
    println("Cout Solution Moyenne (EEV): \$", round(EEV, digits=2))
    println("-"^40)
    println("VSS: \$", round(VSS, digits=2))
    println("-"^40)
else
    println("Error")
end

Set parameter Username
Academic license - for non-commercial use only - expires 2026-11-19
Plan Deterministe: [20.0, 25.0, 25.0, 25.0, 25.0]
Set parameter Username
Academic license - for non-commercial use only - expires 2026-11-19
Cout EEV : 10748.09

----------------------------------------
RÉSULTATS DE LA VSS
----------------------------------------
Cout Solution Stochastique: $10613.39
Cout Solution Moyenne (EEV): $10748.09
----------------------------------------
VSS: $134.7
----------------------------------------


#### Methode 2: Optimisation Robuste  (AARC)

Similairement la formulation vient du travail fait au prealable. 

In [17]:
Random.seed!(21)

T = 5   

S_min, S_max = 10.0, 100.0
R_min, R_max = 0.0, 50.0
S_0          = 50.0
S_objectif   = 30.0

C_X = 10.0   
C_Y = 20.0   
M   = 1000.0 

#Ensemble d'incertitude polyhedrale (geometriquement forme de boite), choisit pour que le probleme soit realisable 
mu_I, sigma_I = 20.0, 5.0
mu_D, sigma_D = 25.0, 5.0
I_min, I_max = mu_I - 2*sigma_I, mu_I + 2*sigma_I #10 => 30
D_min, D_max = mu_D - 2*sigma_D, mu_D + 2*sigma_D #15 =>35

#scenarios extremes sont coins du cubes donc liste pour stocker 
scenarios_coins = []
for i in 1:50
    I_scen = [rand([I_min, I_max]) for _ in 1:T]
    D_scen = [rand([D_min, D_max]) for _ in 1:T]
    push!(scenarios_coins, (I_scen, D_scen))
end

#ajouter des pires scenarios dans le probleme
push!(scenarios_coins, (fill(I_min, T), fill(D_max, T))) #max secheresse
push!(scenarios_coins, (fill(I_max, T), fill(D_min, T))) #max crue


model = Model(Gurobi.Optimizer)
#set_silent(model)

#variable de premiere etape
@variable(model, X[1:T] >= 0) 

#variable de seconde etape
@variable(model, y0[1:T])
@variable(model, Q_I[1:T, 1:T]) 
@variable(model, Q_D[1:T, 1:T]) 

#non anticipativite, beaucoup de difficulte a formule ca pas sure que ce soit la meilleur methode, aide AI pour debug
for t in 1:T
    for k in (t+1):T
        fix(Q_I[t, k], 0.0; force=true)
        fix(Q_D[t, k], 0.0; force=true)
    end
end

#pire cas le cout 
@variable(model, Cout_Pire_Cas >= 0)


for (I_val, D_val) in scenarios_coins
    
    #expression affine
    stock_courant = AffExpr(S_0) #
    cout_scenario = AffExpr(0.0)

    for t in 1:T
        #calcul du recrous affine 
        recours_val = y0[t] + sum(Q_I[t, k] * I_val[k] + Q_D[t, k] * D_val[k] for k in 1:t)
        
        relachement = X[t] + recours_val
        
        #equuilibre du stock a travers les periodes
        stock_courant = stock_courant + I_val[t] - relachement
        
        
        #bornes physiques du reservoir
        @constraint(model, S_min <= stock_courant <= S_max)
        @constraint(model, R_min <= relachement <= R_max)
        
        #satisfaction de la demande
        d_deficit = @variable(model, lower_bound=0) #pour eviter l'infaisabilite du probleme
        @constraint(model, relachement + d_deficit >= D_val[t])
        
        #couts
        abs_Y = @variable(model, lower_bound=0)
        @constraint(model, abs_Y >= recours_val) #mettre les bounds des deux cotes pour lineariser valeur absolue de Y
        @constraint(model, abs_Y >= -recours_val)
        
        add_to_expression!(cout_scenario, C_Y * abs_Y + M * d_deficit)
    end
    
    #objectif d'eau dans le reservoir
    @constraint(model, stock_courant >= S_objectif)
    
    #le pire scenario est oblige d'avoir un cout au minimum egal aux autres scenarios
    @constraint(model, Cout_Pire_Cas >= cout_scenario)
end

#OF
@objective(model, Min, sum(C_X * X[t] for t in 1:T) + Cout_Pire_Cas)


optimize!(model)

#format des solutions utilisations AI
if termination_status(model) == MOI.OPTIMAL
    println("Plan de relachement optimal:")
    println(round.(value.(X), digits=2))
    
    cout_robuste = objective_value(model)
    println("\nValeur Objectif (Cout Robuste) : ", round(cout_robuste, digits=2))
    
    println("\nPire scenario")
    println("(Apports = $I_min, Demande = $D_max)")
    
    #pour le rapport
    curr_S = S_0
    I_sec = fill(I_min, T)
    D_sec = fill(D_max, T)
    
  
    println("Temps| Plan(X)|  Adj(Y)|  Total| Stock")
    for t in 1:T
        y_val = value(y0[t]) + sum(value(Q_I[t,k])*I_sec[k] + value(Q_D[t,k])*D_sec[k] for k in 1:t)
        rel = value(X[t]) + y_val
        curr_S = curr_S + I_sec[t] - rel
        
        @printf("%4d | %6.2f | %6.2f | %6.2f | %6.2f\n", t, value(X[t]), y_val, rel, curr_S)
    end
else
    println("Modèle Infaisable - L'ensemble d'incertitude est peut-être trop large pour les contraintes physiques.")
end

Set parameter Username
Academic license - for non-commercial use only - expires 2026-11-19
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1404 rows, 1101 columns and 16952 nonzeros
Model fingerprint: 0x4dafde4a
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+01, 2e+02]
  RHS range        [2e+01, 2e+02]
Presolve removed 1 rows and 20 columns
Presolve time: 0.01s
Presolved: 1403 rows, 1081 columns, 16912 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0      handle free variables                          0s
    1085    1.0570000e+05   0.000000e+00   0.000000e+00      0s

Use crossover to convert LP symmetric solution to basic solution...
Crossover log...

       0 DPushes remaining with DInf 0.0000000e+00                 0s

     289 PPushes remaining with PInf 0.0000000e+00           

### Comparaison des methodes 

In [18]:
Random.seed!(21)

T = 5   
S_SAA = 50  

S_min, S_max = 10.0, 100.0
R_min, R_max = 0.0, 50.0
S_0        = 50.0  
S_objectif = 30.0

C_X = 10.0  
C_Y = 50.0   
M   = 5000.0 

mu_I_model, sigma_I = 22.0, 4.0  
mu_D, sigma_D       = 20.0, 2.0  

#realite
mu_I_real = 18.0 

#METHODE 1: SAA

I_scenarios = max.(0, mu_I_model .+ sigma_I .* randn(T, S_SAA))
D_scenarios = max.(0, mu_D .+ sigma_D .* randn(T, S_SAA))

model_saa = Model(Gurobi.Optimizer)
set_silent(model_saa) 

@variable(model_saa, X[1:T] >= 0)

@variable(model_saa, u[1:T, 1:S_SAA] >= 0)     
@variable(model_saa, v[1:T, 1:S_SAA] >= 0)     
@variable(model_saa, d[1:T, 1:S_SAA] >= 0)      
@variable(model_saa, Stockage[1:T, 1:S_SAA])    

for s in 1:S_SAA
    for t in 1:T
        prev_S = (t == 1) ? S_0 : Stockage[t-1, s]
        @constraint(model_saa, Stockage[t, s] == prev_S + I_scenarios[t, s] - (X[t] + u[t, s] - v[t, s]))
        @constraint(model_saa, (X[t] + u[t, s] - v[t, s]) + d[t, s] >= D_scenarios[t, s])
        @constraint(model_saa, S_min <= Stockage[t, s] <= S_max)
        current_release = X[t] + u[t, s] - v[t, s]
        @constraint(model_saa, R_min <= current_release <= R_max)
    end
    @constraint(model_saa, Stockage[T, s] >= S_objectif)
end

cout_planifie = sum(C_X * X[t] for t in 1:T)
cout_recours = sum((1/S_SAA) * (C_Y * (u[t, s] + v[t, s]) + M * d[t, s]) for t in 1:T, s in 1:S_SAA)

@objective(model_saa, Min, cout_planifie + cout_recours)

optimize!(model_saa)
X_SAA = value.(X)
println("SAA Solution: ", round.(X_SAA, digits=2))


#METHODE 2: RO (ARRC)

#incertitude U
I_min, I_max = mu_I_model - 2*sigma_I, mu_I_model + 2*sigma_I
D_min, D_max = mu_D - 2*sigma_D, mu_D + 2*sigma_D

scenarios_coins = []
for i in 1:50
    I_scen = [rand([I_min, I_max]) for _ in 1:T]
    D_scen = [rand([D_min, D_max]) for _ in 1:T]
    push!(scenarios_coins, (I_scen, D_scen))
end
push!(scenarios_coins, (fill(I_min, T), fill(D_max, T))) 
push!(scenarios_coins, (fill(I_max, T), fill(D_min, T))) 

model_ro = Model(Gurobi.Optimizer)
set_silent(model_ro)

@variable(model_ro, X_ro[1:T] >= 0) 

@variable(model_ro, y0[1:T])
@variable(model_ro, Q_I[1:T, 1:T]) 
@variable(model_ro, Q_D[1:T, 1:T]) 

for t in 1:T
    for k in (t+1):T
        fix(Q_I[t, k], 0.0; force=true)
        fix(Q_D[t, k], 0.0; force=true)
    end
end

@variable(model_ro, Cout_Pire_Cas >= 0)

for (I_val, D_val) in scenarios_coins
    
    stock_courant = AffExpr(S_0)
    cout_scenario = AffExpr(0.0)

    for t in 1:T
        
        recours_val = y0[t] + sum(Q_I[t, k] * I_val[k] + Q_D[t, k] * D_val[k] for k in 1:t)
        
        relachement = X_ro[t] + recours_val
        
        stock_courant = stock_courant + I_val[t] - relachement
        
        @constraint(model_ro, S_min <= stock_courant <= S_max)
        @constraint(model_ro, R_min <= relachement <= R_max)
    
        d_deficit = @variable(model_ro, lower_bound=0) 
        @constraint(model_ro, relachement + d_deficit >= D_val[t])
        
        abs_Y = @variable(model_ro, lower_bound=0)
        @constraint(model_ro, abs_Y >= recours_val) 
        @constraint(model_ro, abs_Y >= -recours_val)
        
        add_to_expression!(cout_scenario, C_Y * abs_Y + M * d_deficit)
    end
    
    @constraint(model_ro, stock_courant >= S_objectif)
    @constraint(model_ro, Cout_Pire_Cas >= cout_scenario)
end

@objective(model_ro, Min, sum(C_X * X_ro[t] for t in 1:T) + Cout_Pire_Cas)

optimize!(model_ro)
X_RO = value.(X_ro)
println("AARC Optimise: ", round.(X_RO, digits=2))


#VS Realite 
N_SIM = 300 #oblige de faire 300 parce que plus tourne pendant 10+ minutes 

I_sim = max.(0, mu_I_real .+ sigma_I .* randn(T, N_SIM))
D_sim = max.(0, mu_D .+ sigma_D .* randn(T, N_SIM))

#plan fixe, choisir les variables de recours optimale
function simuler(plan, I_vec, D_vec)
    m_sim = Model(Gurobi.Optimizer)
    set_silent(m_sim)
    @variable(m_sim, u[1:T]>=0); @variable(m_sim, v[1:T]>=0); @variable(m_sim, d[1:T]>=0)
    @variable(m_sim, S[1:T])
    
    for t in 1:T
        prev = (t==1) ? S_0 : S[t-1]
        @constraint(m_sim, S[t] == prev + I_vec[t] - (plan[t] + u[t] - v[t]))
        @constraint(m_sim, (plan[t] + u[t] - v[t]) + d[t] >= D_vec[t])
        @constraint(m_sim, S_min <= S[t] <= S_max)
        @constraint(m_sim, R_min <= plan[t] + u[t] - v[t] <= R_max)
    end
    @constraint(m_sim, S[T] >= S_objectif)
    @objective(m_sim, Min, sum(C_Y*(u[t]+v[t]) + M*d[t] for t in 1:T))
    
    optimize!(m_sim)
    if termination_status(m_sim) == MOI.OPTIMAL
        return objective_value(m_sim) + sum(C_X*plan), sum(value.(d)), mean(value.(S))
    else
        return 1e6, 100.0, 0.0 #infaisabilite penalite 
    end
end

res_saa = zeros(N_SIM, 3); res_ro = zeros(N_SIM, 3)
for i in 1:N_SIM
    res_saa[i,:] .= simuler(X_SAA, I_sim[:,i], D_sim[:,i])
    res_ro[i,:]  .= simuler(X_RO,  I_sim[:,i], D_sim[:,i])
end

#pour le rapport 

failures_saa = res_saa[res_saa[:,2] .> 0.01, 2]
failures_ro  = res_ro[res_ro[:,2] .> 0.01, 2]
avg_sev_saa = isempty(failures_saa) ? 0.0 : mean(failures_saa)
avg_sev_ro  = isempty(failures_ro)  ? 0.0 : mean(failures_ro)

#format des solutions utilisation AI 
println("RÉSULTATS FINAUX")
println("-"^60)
@printf("%-20s | %-15s | %-15s\n", "METRIQUE", "SAA", "RO")
println("-"^60)
@printf("%-20s | \$%-14.2f | \$%-14.2f\n", "Cout Total Moyen", mean(res_saa[:,1]), mean(res_ro[:,1]))
@printf("%-20s | \$%-14.2f | \$%-14.2f\n", "Cout Pire Cas", maximum(res_saa[:,1]), maximum(res_ro[:,1]))
@printf("%-20s | %-15.2f | %-15.2f\n", "Vol. Stock Moyen", mean(res_saa[:,3]), mean(res_ro[:,3]))
println("-"^60)

Set parameter Username
Academic license - for non-commercial use only - expires 2026-11-19
SAA Solution: [21.8, 21.93, 21.54, 21.81, 21.46]
Set parameter Username
Academic license - for non-commercial use only - expires 2026-11-19
AARC Optimise: [24.0, 24.0, 20.0, 16.0, 6.0]
Set parameter Username
Academic license - for non-commercial use only - expires 2026-11-19
Set parameter Username
Academic license - for non-commercial use only - expires 2026-11-19
Set parameter Username
Academic license - for non-commercial use only - expires 2026-11-19
Set parameter Username
Academic license - for non-commercial use only - expires 2026-11-19
Set parameter Username
Academic license - for non-commercial use only - expires 2026-11-19
Set parameter Username
Academic license - for non-commercial use only - expires 2026-11-19
Set parameter Username
Academic license - for non-commercial use only - expires 2026-11-19
Set parameter Username
Academic license - for non-commercial use only - expires 2026-11

Le scenario precedent simulait des secheresses massives. La difference entre les deux problemes et petites simplement parc eque le probleme est physiquement impossible dans les deux cas. Quand l'influx d'eau est tres faible la difference entre SAA et RO disparait car dans les deux cas il n'y a pas assez d'eau. Pour autant, on voit que le cout moyen est plus eleve pour l'optimisation robuste. Cela est le prix de son conservatisme. Aussi, le prix dans le pire des cas est superieur pour la methode SAA. 